In [1]:
#Script to obtain data 

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

#Libraries to create the multiclass model
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
#Import tensorflow and disable the v2 behavior and eager mode
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
tf.compat.v1.disable_v2_behavior()

#Library to validate the model
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score

Instructions for updating:
non-resource variables are not supported in the long term


In [203]:
df = pd.read_excel("csv/label.xlsx")

In [204]:
df

,Name,Artists,Track_id,Album,Release Date,Image_URL,Danceability,Energy,Loudness,Speechiness,Acousticness,Tempo,Liveness,Valence,mood
0,Hosanna,"A.R. Rahman,Vijay Prakash,Suzanne D'Mello,Blaaze",3l7JaDIJmWMpvcJqhTJjrk,Vinnathaandi Varuvaayaa (Original Motion Pictu...,2010-01-08 00:00:00,https://i.scdn.co/image/ab67616d00001e0297838a...,0.582,0.812,-6.147,0.0418,0.166,86.051,0.0407,0.7000,calm
1,Mazhai Varum - Female,"Joshua Sridhar,Suzanne D'Mello",6sewH6rrJZeM40FdCxbJP8,Veppam (Original Motion Picture Soundtrack),2011-01-06 00:00:00,https://i.scdn.co/image/ab67616d00001e028179a9...,0.697,0.690,-4.732,0.0302,0.188,95.014,0.1560,0.5860,calm
2,Oru Kili Oru Kili,"Satish Chakravarthy,Shreya Ghoshal",4KeY07xAKNv7hftaTRaqy4,Leelai (Original Motion Picture Soundtrack),2009-08-18 00:00:00,https://i.scdn.co/image/ab67616d00001e0245c1b7...,0.602,0.614,-7.090,0.0434,0.249,113.986,0.3440,0.1920,calm
3,Kaatrukullai,Yuvan Shankar Raja,3tOR1zae8trHZweUByE0Zv,Sarvam,2009-02-14 00:00:00,https://i.scdn.co/image/ab67616d00001e028f83ad...,0.805,0.588,-5.786,0.0311,0.431,119.990,0.1180,0.5180,calm
4,Machaan Machaan,"Yuvan Shankar Raja,Ilaiyaraaja,Bela Shende",096wyG5zJ7oUaZqOCGmmqH,Silambattam (Original Motion Picture Soundtrack),2008-10-25 00:00:00,https://i.scdn.co/image/ab67616d00001e02794a0b...,0.627,0.682,-10.807,0.0895,0.331,173.968,0.1270,0.8170,calm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1061,Poi Vazhva,"Santhosh Narayanan,Vijaynarain",3JHqj3WfflTssLpj1Ymsw2,Manithan (Original Motion Picture Soundtrack),2016-04-08 00:00:00,https://i.scdn.co/image/ab67616d00001e02803e99...,0.694,0.425,-10.189,0.0349,0.630,118.869,0.1820,0.3480,sad
1062,Dhimu Dhimu,"Harris Jayaraj,Karthik",6old0mA1IHi5lw2yLv3ri6,Engeyum Kadhal (Original Motion Picture Soundt...,2010-11-11 00:00:00,https://i.scdn.co/image/ab67616d00001e02340026...,0.643,0.478,-9.376,0.0274,0.569,93.418,0.1180,0.3170,sad
1063,Madura Marikozhundhae,"A.R. Rahman,Anuradha Sriram,Shweta Mohan,Aparn...",4zfZi4L6lsf152iSWZo40Q,Chekka Chivantha Vaanam (Original Motion Pictu...,2018-09-28 00:00:00,https://i.scdn.co/image/ab67616d00001e025234d8...,0.238,0.372,-12.117,0.0316,0.545,70.674,0.0840,0.0703,sad
1064,Poi Vaada,Srimathumitha,7KQzBQAJShO2Eik8SUjOJT,Dharmadurai (Original Motion Picture Soundtrack),2016-08-02 00:00:00,https://i.scdn.co/image/ab67616d00001e02ae45c8...,0.366,0.330,-7.198,0.0356,0.278,89.108,0.1090,0.1260,sad


In [119]:
df.describe()

,Energy,Valence,label
count,1066.000000,1066.000000,1066.000000
mean,0.631994,0.568313,1.515947
std,0.196439,0.218787,1.095543
min,0.095300,0.035900,0.000000
25%,0.491000,0.407000,1.000000
50%,0.641000,0.582000,1.000000
75%,0.794000,0.746000,2.000000
max,0.985000,0.972000,3.000000


In [250]:

X2 = np.array(df.loc[:,['Danceability','Energy','Valence']])
Y = df['mood']
X2

array([[0.582 , 0.812 , 0.7   ],
       [0.697 , 0.69  , 0.586 ],
       [0.602 , 0.614 , 0.192 ],
       ...,
       [0.238 , 0.372 , 0.0703],
       [0.366 , 0.33  , 0.126 ],
       [0.676 , 0.52  , 0.631 ]])

In [251]:
#Encodethe categories
encoder = LabelEncoder()
encoder.fit(Y)
encoded_y = encoder.transform(Y)


#Convert to  dummy (Not necessary in my case)
dummy_y = np_utils.to_categorical(encoded_y)

X_train,X_test,Y_train,Y_test = train_test_split(X2,encoded_y,test_size=0.2,random_state=15)

target = pd.DataFrame({'mood':df['mood'].tolist(),'encode':encoded_y}).drop_duplicates().sort_values(['encode'],ascending=True)
X_train

array([[0.81 , 0.6  , 0.836],
       [0.853, 0.668, 0.648],
       [0.736, 0.778, 0.837],
       ...,
       [0.75 , 0.691, 0.401],
       [0.768, 0.814, 0.748],
       [0.715, 0.497, 0.396]])

In [175]:
X2.size

2132

In [252]:
target

,mood,encode
0,calm,0
337,energy,1
594,happy,2
868,sad,3


In [261]:
def base_model():
    #Create the model
    model = Sequential()
    #Add 1 layer with 8 nodes,input of 4 dim with relu function
    model.add(Dense(4,input_dim=3,activation='relu'))
    model.add(Dense(8,input_dim=4,activation='relu'))
    #Add 1 layer with output 3 and softmax function
    model.add(Dense(4,activation='softmax'))
    #Compile the model using sigmoid loss function and adam optim
    model.compile(loss='categorical_crossentropy',optimizer='adam',
                 metrics=['accuracy'])
    return model

In [262]:
estimator = KerasClassifier(build_fn=base_model,epochs=300,batch_size=128,verbose=1)

C:\Users\surya\AppData\Local\Temp\ipykernel_5428\2776211349.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=base_model,epochs=300,batch_size=128,verbose=1)


In [263]:
estimator.fit(X_train,Y_train)
y_preds = estimator.predict(X_test)


Train on 852 samples
Epoch 1/300
852/852 [==============================] - 1s 809us/sample - loss: 1.3907 - acc: 0.2500
Epoch 2/300
852/852 [==============================] - 0s 161us/sample - loss: 1.3867 - acc: 0.2500
Epoch 3/300
852/852 [==============================] - 0s 109us/sample - loss: 1.3837 - acc: 0.2500
Epoch 4/300
852/852 [==============================] - 0s 106us/sample - loss: 1.3813 - acc: 0.2547
Epoch 5/300
852/852 [==============================] - 0s 146us/sample - loss: 1.3798 - acc: 0.2054
Epoch 6/300
852/852 [==============================] - 0s 92us/sample - loss: 1.3782 - acc: 0.2347
Epoch 7/300
852/852 [==============================] - 0s 106us/sample - loss: 1.3771 - acc: 0.2923
Epoch 8/300
852/852 [==============================] - 0s 93us/sample - loss: 1.3759 - acc: 0.3134
Epoch 9/300
852/852 [==============================] - 0s 104us/sample - loss: 1.3748 - acc: 0.2958
Epoch 10/300
852/852 [==============================] - 0s 112us/sample - loss: 1

852/852 [==============================] - 0s 171us/sample - loss: 1.3102 - acc: 0.3521
Epoch 83/300
852/852 [==============================] - 0s 120us/sample - loss: 1.3087 - acc: 0.3509
Epoch 84/300
852/852 [==============================] - 0s 132us/sample - loss: 1.3071 - acc: 0.3521
Epoch 85/300
852/852 [==============================] - 0s 94us/sample - loss: 1.3056 - acc: 0.3545
Epoch 86/300
852/852 [==============================] - 0s 249us/sample - loss: 1.3041 - acc: 0.3533
Epoch 87/300
852/852 [==============================] - 0s 135us/sample - loss: 1.3024 - acc: 0.3545
Epoch 88/300
852/852 [==============================] - 0s 209us/sample - loss: 1.3008 - acc: 0.3545
Epoch 89/300
852/852 [==============================] - 0s 180us/sample - loss: 1.2991 - acc: 0.3592
Epoch 90/300
852/852 [==============================] - 0s 217us/sample - loss: 1.2975 - acc: 0.3568
Epoch 91/300
852/852 [==============================] - 0s 178us/sample - loss: 1.2960 - acc: 0.3615
Epoc

Epoch 163/300
852/852 [==============================] - 0s 108us/sample - loss: 1.1758 - acc: 0.4413
Epoch 164/300
852/852 [==============================] - 0s 110us/sample - loss: 1.1748 - acc: 0.4460
Epoch 165/300
852/852 [==============================] - 0s 110us/sample - loss: 1.1737 - acc: 0.4472
Epoch 166/300
852/852 [==============================] - 0s 217us/sample - loss: 1.1725 - acc: 0.4425
Epoch 167/300
852/852 [==============================] - 0s 129us/sample - loss: 1.1715 - acc: 0.4401
Epoch 168/300
852/852 [==============================] - 0s 143us/sample - loss: 1.1706 - acc: 0.4425
Epoch 169/300
852/852 [==============================] - 0s 108us/sample - loss: 1.1692 - acc: 0.4413
Epoch 170/300
852/852 [==============================] - 0s 80us/sample - loss: 1.1687 - acc: 0.4460
Epoch 171/300
852/852 [==============================] - 0s 65us/sample - loss: 1.1674 - acc: 0.4460
Epoch 172/300
852/852 [==============================] - 0s 101us/sample - loss: 1.1

852/852 [==============================] - 0s 175us/sample - loss: 1.1265 - acc: 0.4425
Epoch 244/300
852/852 [==============================] - 0s 75us/sample - loss: 1.1259 - acc: 0.4437
Epoch 245/300
852/852 [==============================] - 0s 70us/sample - loss: 1.1253 - acc: 0.4448
Epoch 246/300
852/852 [==============================] - 0s 87us/sample - loss: 1.1250 - acc: 0.4413
Epoch 247/300
852/852 [==============================] - 0s 80us/sample - loss: 1.1248 - acc: 0.4413
Epoch 248/300
852/852 [==============================] - 0s 93us/sample - loss: 1.1246 - acc: 0.4413
Epoch 249/300
852/852 [==============================] - 0s 73us/sample - loss: 1.1243 - acc: 0.4413
Epoch 250/300
852/852 [==============================] - 0s 103us/sample - loss: 1.1242 - acc: 0.4401
Epoch 251/300
852/852 [==============================] - 0s 89us/sample - loss: 1.1239 - acc: 0.4448
Epoch 252/300
852/852 [==============================] - 0s 76us/sample - loss: 1.1236 - acc: 0.4460
Ep

In [233]:
new_df = pd.read_csv("csv/energy1.csv", encoding = 'ISO-8859-1')

In [234]:
new_df

,Name,Artists,Track_id,Album,Release Date,Image_URL,Danceability,Energy,Loudness,Speechiness,Acousticness,Tempo,Liveness,Valence
0,Vilayaadu Mankatha - Extended Dance Mix,"Yuvan Shankar Raja,Premgi Amaren",18gHDBhhfYzCseYrnTZUso,Mankatha (Original Motion Picture Soundtrack),2011-08-10,https://i.scdn.co/image/ab67616d00001e0224f3ca...,0.773,0.847,-5.509,0.0357,0.00265,129.992,0.8150,0.569
1,Otha Sollaala,"G. V. Prakash,Velmurugan",7lnRiEP7BHXuILXNY9MH8f,Aadukalam (Original Motion Picture Soundtrack),2010-11-08,https://i.scdn.co/image/ab67616d00001e02e9310d...,0.707,0.925,-5.675,0.1850,0.68100,79.932,0.0748,0.956
2,Munnal Kadhali,"D. Imman,Vishal Dadlani,Sharanya Gopinath",4yiRdnfoc94dU7dd41YzrX,Miruthan (Original Motion Picture Soundtrack),2016-01-09,https://i.scdn.co/image/ab67616d00001e029ff7b9...,0.558,0.705,-4.149,0.0340,0.07790,173.987,0.1360,0.428
3,"Pondatti (From ""Osthe"")","Thaman S,Silambarasan TR",5cz2kttdC0ZjgApXHpGFer,Playback: Kaadhal Galatta - Fun Tamil Love Songs,2015-02-12,https://i.scdn.co/image/ab67616d00001e02011454...,0.857,0.797,-7.756,0.0688,0.66700,125.012,0.1050,0.825
4,Thangamey,Anirudh Ravichander,5b1zcET4JFw1BHh8rsjHq3,Wunderboy Anirudh,2016-03-03,https://i.scdn.co/image/ab67616d00001e02947069...,0.583,0.901,-5.785,0.0739,0.48000,99.927,0.2480,0.618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"Mama Mama - From ""Boss Engira Baskaran""","Vijay Yesudas,Swetha",1hSLQTWi5io7Doq8h9vPcV,"Party Beats, Vol. 4",2014-11-17,https://i.scdn.co/image/ab67616d00001e02f719bf...,0.772,0.859,-4.281,0.1130,0.40000,79.953,0.0701,0.406
96,"Karuthavanlaam Galeejaam (From ""Velaikkaran"")",Anirudh Ravichander,7vgWggk85sh6n8iMmMDZVJ,"Karuthavanlaam Galeejaam (From ""Velaikkaran"")",2017-08-28,https://i.scdn.co/image/ab67616d00001e027e4dda...,0.670,0.940,-4.404,0.3080,0.20900,89.950,0.3460,0.474
97,Maatikichu,V.M. Mahalingam,402tkgRtRsBEv9roh8Frhk,Meesaya Murukku (Original Motion Picture Sound...,2017-07-28,https://i.scdn.co/image/ab67616d00001e02542a1f...,0.545,0.985,-2.272,0.2890,0.04860,180.014,0.2690,0.871
98,Tasakku Tasakku,"Mukesh,M.L.R. Karthikeyan,Guna",3x5GBpM7Dy4gckH5a2VN0f,Vikram Vedha (Original Motion Picture Soundtrack),2017-06-19,https://i.scdn.co/image/ab67616d00001e029cd6fd...,0.643,0.763,-6.771,0.5860,0.36700,107.049,0.0430,0.880


In [236]:
arr=new_df.iloc[:,6:]

arr

,Danceability,Energy,Loudness,Speechiness,Acousticness,Tempo,Liveness,Valence
0,0.773,0.847,-5.509,0.0357,0.00265,129.992,0.8150,0.569
1,0.707,0.925,-5.675,0.1850,0.68100,79.932,0.0748,0.956
2,0.558,0.705,-4.149,0.0340,0.07790,173.987,0.1360,0.428
3,0.857,0.797,-7.756,0.0688,0.66700,125.012,0.1050,0.825
4,0.583,0.901,-5.785,0.0739,0.48000,99.927,0.2480,0.618
...,...,...,...,...,...,...,...,...
95,0.772,0.859,-4.281,0.1130,0.40000,79.953,0.0701,0.406
96,0.670,0.940,-4.404,0.3080,0.20900,89.950,0.3460,0.474
97,0.545,0.985,-2.272,0.2890,0.04860,180.014,0.2690,0.871
98,0.643,0.763,-6.771,0.5860,0.36700,107.049,0.0430,0.880


In [237]:
pred = estimator.predict(arr)
print(pred)

[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]


In [150]:
df

,Name,Artists,Track_id,Album,Release Date,Image_URL,Danceability,Energy,Loudness,Speechiness,Acousticness,Tempo,Liveness,Valence,mood
0,Hosanna,"A.R. Rahman,Vijay Prakash,Suzanne D'Mello,Blaaze",3l7JaDIJmWMpvcJqhTJjrk,Vinnathaandi Varuvaayaa (Original Motion Pictu...,2010-01-08 00:00:00,https://i.scdn.co/image/ab67616d00001e0297838a...,0.582,0.812,-6.147,0.0418,0.166,86.051,0.0407,0.7000,calm
1,Mazhai Varum - Female,"Joshua Sridhar,Suzanne D'Mello",6sewH6rrJZeM40FdCxbJP8,Veppam (Original Motion Picture Soundtrack),2011-01-06 00:00:00,https://i.scdn.co/image/ab67616d00001e028179a9...,0.697,0.690,-4.732,0.0302,0.188,95.014,0.1560,0.5860,calm
2,Oru Kili Oru Kili,"Satish Chakravarthy,Shreya Ghoshal",4KeY07xAKNv7hftaTRaqy4,Leelai (Original Motion Picture Soundtrack),2009-08-18 00:00:00,https://i.scdn.co/image/ab67616d00001e0245c1b7...,0.602,0.614,-7.090,0.0434,0.249,113.986,0.3440,0.1920,calm
3,Kaatrukullai,Yuvan Shankar Raja,3tOR1zae8trHZweUByE0Zv,Sarvam,2009-02-14 00:00:00,https://i.scdn.co/image/ab67616d00001e028f83ad...,0.805,0.588,-5.786,0.0311,0.431,119.990,0.1180,0.5180,calm
4,Machaan Machaan,"Yuvan Shankar Raja,Ilaiyaraaja,Bela Shende",096wyG5zJ7oUaZqOCGmmqH,Silambattam (Original Motion Picture Soundtrack),2008-10-25 00:00:00,https://i.scdn.co/image/ab67616d00001e02794a0b...,0.627,0.682,-10.807,0.0895,0.331,173.968,0.1270,0.8170,calm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1061,Poi Vazhva,"Santhosh Narayanan,Vijaynarain",3JHqj3WfflTssLpj1Ymsw2,Manithan (Original Motion Picture Soundtrack),2016-04-08 00:00:00,https://i.scdn.co/image/ab67616d00001e02803e99...,0.694,0.425,-10.189,0.0349,0.630,118.869,0.1820,0.3480,sad
1062,Dhimu Dhimu,"Harris Jayaraj,Karthik",6old0mA1IHi5lw2yLv3ri6,Engeyum Kadhal (Original Motion Picture Soundt...,2010-11-11 00:00:00,https://i.scdn.co/image/ab67616d00001e02340026...,0.643,0.478,-9.376,0.0274,0.569,93.418,0.1180,0.3170,sad
1063,Madura Marikozhundhae,"A.R. Rahman,Anuradha Sriram,Shweta Mohan,Aparn...",4zfZi4L6lsf152iSWZo40Q,Chekka Chivantha Vaanam (Original Motion Pictu...,2018-09-28 00:00:00,https://i.scdn.co/image/ab67616d00001e025234d8...,0.238,0.372,-12.117,0.0316,0.545,70.674,0.0840,0.0703,sad
1064,Poi Vaada,Srimathumitha,7KQzBQAJShO2Eik8SUjOJT,Dharmadurai (Original Motion Picture Soundtrack),2016-08-02 00:00:00,https://i.scdn.co/image/ab67616d00001e02ae45c8...,0.366,0.330,-7.198,0.0356,0.278,89.108,0.1090,0.1260,sad


In [81]:
new_list=[]
for i , j in df.iterrows():
    if j[0] == 0:
        new_list.append('Calm')
    elif j[0] == 1:
        new_list.append('Energitic')
    elif j[0] == 2:
        new_list.append('Happy')
    else:
        new_list.append('Sad')
new_list

['Happy',
 'Energitic',
 'Energitic',
 'Energitic',
 'Happy',
 'Energitic',
 'Happy',
 'Sad',
 'Happy',
 'Happy',
 'Energitic',
 'Happy',
 'Energitic',
 'Energitic',
 'Energitic',
 'Energitic',
 'Energitic',
 'Calm',
 'Sad',
 'Calm',
 'Sad',
 'Energitic',
 'Energitic',
 'Happy',
 'Happy',
 'Energitic',
 'Energitic',
 'Energitic',
 'Calm',
 'Sad',
 'Sad',
 'Calm',
 'Energitic',
 'Energitic',
 'Sad',
 'Calm',
 'Happy',
 'Energitic',
 'Calm',
 'Energitic',
 'Calm',
 'Calm',
 'Sad',
 'Calm',
 'Happy',
 'Calm',
 'Sad',
 'Energitic',
 'Sad',
 'Energitic',
 'Energitic',
 'Calm',
 'Happy',
 'Energitic',
 'Calm',
 'Energitic',
 'Calm',
 'Sad',
 'Sad',
 'Energitic',
 'Energitic',
 'Sad',
 'Energitic',
 'Energitic',
 'Happy',
 'Energitic',
 'Energitic',
 'Happy',
 'Happy',
 'Energitic',
 'Calm',
 'Calm',
 'Energitic',
 'Sad',
 'Happy',
 'Calm',
 'Calm',
 'Calm',
 'Calm',
 'Sad',
 'Happy',
 'Calm',
 'Energitic',
 'Sad',
 'Sad',
 'Calm',
 'Calm',
 'Sad',
 'Energitic',
 'Calm',
 'Happy',
 'Sad',
 'C

In [83]:
new_list

['Happy',
 'Energitic',
 'Energitic',
 'Energitic',
 'Happy',
 'Energitic',
 'Happy',
 'Sad',
 'Happy',
 'Happy',
 'Energitic',
 'Happy',
 'Energitic',
 'Energitic',
 'Energitic',
 'Energitic',
 'Energitic',
 'Calm',
 'Sad',
 'Calm',
 'Sad',
 'Energitic',
 'Energitic',
 'Happy',
 'Happy',
 'Energitic',
 'Energitic',
 'Energitic',
 'Calm',
 'Sad',
 'Sad',
 'Calm',
 'Energitic',
 'Energitic',
 'Sad',
 'Calm',
 'Happy',
 'Energitic',
 'Calm',
 'Energitic',
 'Calm',
 'Calm',
 'Sad',
 'Calm',
 'Happy',
 'Calm',
 'Sad',
 'Energitic',
 'Sad',
 'Energitic',
 'Energitic',
 'Calm',
 'Happy',
 'Energitic',
 'Calm',
 'Energitic',
 'Calm',
 'Sad',
 'Sad',
 'Energitic',
 'Energitic',
 'Sad',
 'Energitic',
 'Energitic',
 'Happy',
 'Energitic',
 'Energitic',
 'Happy',
 'Happy',
 'Energitic',
 'Calm',
 'Calm',
 'Energitic',
 'Sad',
 'Happy',
 'Calm',
 'Calm',
 'Calm',
 'Calm',
 'Sad',
 'Happy',
 'Calm',
 'Energitic',
 'Sad',
 'Sad',
 'Calm',
 'Calm',
 'Sad',
 'Energitic',
 'Calm',
 'Happy',
 'Sad',
 'C

In [84]:
new_df['emotion'] = new_list


In [85]:
new_df

,Name,Artists,Track_id,Album,Release Date,Image_URL,Danceability,Energy,Loudness,Speechiness,Acousticness,Tempo,Liveness,Valence,mood,emotion
0,Hosanna,"A.R. Rahman,Vijay Prakash,Suzanne D'Mello,Blaaze",3l7JaDIJmWMpvcJqhTJjrk,Vinnathaandi Varuvaayaa (Original Motion Pictu...,08-01-2010,https://i.scdn.co/image/ab67616d00001e0297838a...,0.582,0.812,-6.147,0.0418,0.166,86.051,0.0407,0.700,calm,Happy
1,Mazhai Varum - Female,"Joshua Sridhar,Suzanne D'Mello",6sewH6rrJZeM40FdCxbJP8,Veppam (Original Motion Picture Soundtrack),06-01-2011,https://i.scdn.co/image/ab67616d00001e028179a9...,0.697,0.690,-4.732,0.0302,0.188,95.014,0.1560,0.586,calm,Energitic
2,Oru Kili Oru Kili,"Satish Chakravarthy,Shreya Ghoshal",4KeY07xAKNv7hftaTRaqy4,Leelai (Original Motion Picture Soundtrack),18-08-2009,https://i.scdn.co/image/ab67616d00001e0245c1b7...,0.602,0.614,-7.090,0.0434,0.249,113.986,0.3440,0.192,calm,Energitic
3,Kaatrukullai,Yuvan Shankar Raja,3tOR1zae8trHZweUByE0Zv,Sarvam,14-02-2009,https://i.scdn.co/image/ab67616d00001e028f83ad...,0.805,0.588,-5.786,0.0311,0.431,119.990,0.1180,0.518,calm,Energitic
4,Machaan Machaan,"Yuvan Shankar Raja,Ilaiyaraaja,Bela Shende",096wyG5zJ7oUaZqOCGmmqH,Silambattam (Original Motion Picture Soundtrack),25-10-2008,https://i.scdn.co/image/ab67616d00001e02794a0b...,0.627,0.682,-10.807,0.0895,0.331,173.968,0.1270,0.817,calm,Happy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,Mudhal Murai,"Srinivas,Sujatha",3GW7KmMlsYrQgMQ3O4Rlc8,Sangamam (Original Motion Picture Soundtrack),16-07-1999,https://i.scdn.co/image/ab67616d00001e0286edc1...,0.466,0.600,-8.735,0.1370,0.194,90.314,0.1280,0.529,calm,Energitic
333,Margazhi Thingal Allava,"P. Unnikrishnan, Srimathumitha,S. Janaki",5EBWltYF8zivuKoWMfbajP,Sangamam (Original Motion Picture Soundtrack),16-07-1999,https://i.scdn.co/image/ab67616d00001e0286edc1...,0.402,0.717,-6.540,0.0745,0.532,98.197,0.0993,0.216,calm,Energitic
334,Sowkiyama Kannae,Nithyasree Mahadevan,0osBITP5sDO4jEFNqquQS2,Sangamam (Original Motion Picture Soundtrack),16-07-1999,https://i.scdn.co/image/ab67616d00001e0286edc1...,0.654,0.744,-7.404,0.0473,0.252,99.992,0.1190,0.354,calm,Energitic
335,O Vennilla,Kadhal Desam,68YRDduKLy0DFKp2RWd9VS,Kadhal Desam,01-10-2007,https://i.scdn.co/image/ab67616d00001e02d1595e...,0.498,0.193,-16.050,0.0305,0.536,131.143,0.0836,0.116,calm,Calm
